#### _Impact du confinement durant la crise du COVID-19 sur le niveau de bruit sismique ambiant._

# VARIATIONS TEMPORELLES DU BRUIT SELON LES MESURES SANITAIRES EN VIGUEUR

---

**Description du Notebook :**
> Dans ce notebook est étudié comment le bruit sismique à évolué avec les différentes mesures sanitaires mises en place par le gouvernement.


---
# Importation des modules

In [ ]:
%matplotlib inline

import datetime
import os
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
import warnings

from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.signal import PPSD

%load_ext autoreload
%autoreload 2
from seismic_noise import *
import seismosocialdistancing

---
# 1. Récupération des données

### Paramètres initiaux

In [ ]:
### Paramètres généraux
load_data = False              # Recalculer les PPSDs et le déplacement RMS
# freqs = [(4.0,14.0)]          # Bandes de fréquences à étudier 
st_file = "DATA/st_metadata/stations_fr.txt"  # Fichier de stations
noise_input_data = "B:/SeismicNoiseData/"
timeseries_data = "Files/noise_timeseries/"
PPSD_FOLDER  = "/media/flavien/Stockage/SeismicNoiseData/PPSD"
DRMS_FOLDER  = "/media/flavien/Stockage/SeismicNoiseData/DRMS"
MSEED_FOLDER = "/media/flavien/Stockage/SeismicNoiseData/MSEED"
FIG_FOLDER = "RESULTS"
tStart = "2020-01-01"
tEnd = "2021-08-01"

### Conversion des périodes en UTCDateTime
periods = [[UTCDateTime(tStart), UTCDateTime(tEnd)]]

### Récupération des PPSDs et du déplacement RMS

In [ ]:
st_name = np.loadtxt(st_file, dtype=str)
# st_name = ["FR.STR.00.HHZ"]   # Station à étudier
dRMS = {}
for st_str in st_name:
    try:
        dataframe = pd.read_csv("{}/{}.csv".format(DRMS_FOLDER, st_str), index_col=0)
        dRMS[st_str] = dataframe
    except:
        pass
st_name = list(dRMS.keys())

In [ ]:
dRMS.keys()

---
# 2. Variations horaires du bruit

## 2.1 Calcul de variations sur toute la période donnée

#### Tags à afficher sur le graphique

In [ ]:
tags = [
    ("2020-03-17", 6, "Confinement 1"),
    ("2020-05-11", 6, "Déconfinement"),
    ("2020-10-30", 6, "Confinement 2"),
    ("2020-12-15", 20, "Déconfinement + Couvre-feu 20h"),
    ("2021-01-10", 18, "Couvre feu à 18h"),
    ("2021-04-03", 6, "Confinement 3"),
    ("2021-05-03", 6, "Déconfinement"),
    ("2021-06-09", 23, "Couvre feu à 23h"),
    ("2021-06-30", 6, "Fin du couvre feu")
]

In [ ]:
### Calcul de la variation du bruit
k = 1
for st_str in st_name:
    print("{}/{}    {}".format(k, len(st_name), st_str))
    for freq in dRMS[st_str].columns:

        try:
            noise = dRMS[st_str][freq]  # Série temporelle du bruit
        except:
            continue
        t, h, noise_matrix = hourly_var(noise, periods)
        grid = noise_matrix * 1e9  # Données en [nm]

        ### Mise en graphique de la variation du bruit
        plt.figure(figsize=(16, 8))
        plt.pcolormesh(t, h+0.25, grid)
        plt.clim(np.nanquantile(grid, 0.05), np.nanquantile(grid, 0.95))
        plt.colorbar(label="Déplacement [nm]")

        ### Affichage des tags
        for (date, hour, text) in tags:
            # plt.scatter(np.datetime64(date), hour, zorder=50, s=40, color="red", edgecolors="white")
            plt.axvline(np.datetime64(date), zorder=50, color="white", ls="-")
            plt.text(np.datetime64(date)-8, 0.75, text, color="white", fontsize="medium", rotation=90)   # Confinement 1

        ### Configuration de l'axe vertical
        plt.ylabel("Heure locale")
        y_ticks = ["%ih" % j for j in range(1, 24)]
        y_loc = [j for j in range(1, 24)]
        plt.yticks(y_loc, y_ticks)
        plt.title("{} - [{}] Hz".format(st_str, freq))
        folder_path = FIG_FOLDER + "/fr_hourly_variations/" + st_str

        if not os.path.exists(folder_path): os.makedirs(folder_path)
        plt.savefig("{}/hourly_temporal_{}.png".format(folder_path, freq), dpi=150)
        plt.close()
        plt.clf()
        
    k += 1

## 2.2 Calcul de variations horaires sur plusieurs sous-périodes

In [ ]:
subperiods = [
    ["2020-10-01", "2020-10-20", "No lockdown", "#8F8F8F"],
    ["2020-03-18", "2020-05-10", "Lockdown 1", "#3D80B0"],
    ["2020-11-01", "2020-12-10", "Lockdown 2", "#C53F2C"],
    ["2020-12-10", "2020-12-31", "Christmas + Curfew 20h-6h", "#299739"],
    ["2021-01-16", "2021-02-16", "Curfew 18h-6h", "#902F8C"],
    ["2021-04-03", "2021-05-03", "Lockdown 3", "#E59B1A"],
    ["2021-05-03", "2021-06-09", "Crufew 19h", "green"],
    ["2021-06-09", "2021-06-30", "Crufew 23h", "blue"],
    ["2021-06-30", "2021-07-31", "No curfew", "red"]
]

In [ ]:
k = 1

for st_str in st_name:
    print("{}/{}    {}".format(k, len(st_name), st_str))
    for freq in dRMS[st_str].columns:

        try:
            noise = dRMS[st_str][freq]  # Série temporelle du bruit
        except:
            continue

        t, h, noise_matrix = hourly_var(noise, periods)
        grid = noise_matrix * 1e9  # Données en [nm]

        ### Calcul du bruit pour que sous-période
        plt.figure(figsize=(12, 6))
        for i, value in enumerate(subperiods):
            time1 = np.datetime64(subperiods[i][0])
            time2 = np.datetime64(subperiods[i][1])
            color = subperiods[i][3]
            label = "{} to {} : {}".format(subperiods[i][0], subperiods[i][1], subperiods[i][2])
            k1 = np.where(t == time1)[0][0]
            k2 = np.where(t == time2)[0][0]

            median = np.nanmedian(grid[:,k1:k2], axis=1)
            if subperiods[i][2]=="No lockdown": plt.plot(h, median, label=label, color=color, lw=2, ls="--")
            else: plt.plot(h, median, label=label, color=color, lw=2)

        ### Mise en graphique
        plt.tick_params(direction="in", which="both", top=True, right=True)
        plt.title("{} - [{}] Hz".format(st_str,freq))
        plt.grid(color="#D5D5D5", linestyle='--', linewidth=1)
        # plt.ylabel("Median displacement [nm]")
        plt.xlim([0, 23.5])
        plt.xticks(np.arange(0, 23, 2), ["0h","2h","4h","6h","8h","10h","12h","14h","16h","18h","20h","22h"])

        plt.legend(loc="upper left",   # Position of legend
                   borderaxespad=0.2,    # Small spacing around legend box
                   bbox_to_anchor=(-0.1, -0.1),
                   ncol=3)

        plt.tight_layout()

        folder_path = FIG_FOLDER + "/fr_hourly_variations/" + st_str

        if not os.path.exists(folder_path): os.makedirs(folder_path)

        plt.savefig("{}/hourly_periods_{}.png".format(folder_path, freq), dpi=150)
        plt.close()
        plt.clf()

    k += 1